In [1]:
import os
import argparse
import logging
import time
import numpy as np
import numpy.random as npr
# import matplotlib
# matplotlib.use('agg')
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import string
import gc, os, shutil

from modules import *

## Obtain Brill-Lindquist

In [ ]:
y_init = -1
y_final = 1
x_bounds = (-0.5, 0.5)
z_bounds = (-0.5, 0.5)


# black hole configuration
BH_res = 3
positions = []

masses = np.zeros((BH_res, BH_res, BH_res))

Layer1_center = (1, 1, 2)  # Layer1 center
Layer2_top_row_center = (1, 0, 1)  # Layer2 top row center
Layer2_center = (1, 1, 1)  # Layer2 center
Layer2_bottom_left = (0, 2, 1)  # Layer2 bottom-left
Layer2_bottom_right = (2, 2, 1)  # Layer2 bottom-right
Layer3_center = (1, 1, 0)  # Layer3 center

# masses[Layer1_center] = 0.01
# masses[Layer3_center] = 0.05
masses[Layer2_center] = 0.07
# masses[Layer2_top_row_center] = 0.04
masses[Layer2_bottom_left] = 0.2
masses[Layer2_bottom_right] = 0.1

masses = masses.flatten()
# masses = masses_presoftplus + np.abs(np.random.normal(loc=0, scale=5, size=masses_presoftplus.shape))

x_coords = np.linspace(-0.5, 0.5, BH_res)
y_coords = np.linspace(-0.5, 0.5, BH_res)
z_coords = np.linspace(-0.5, 0.5, BH_res)   
for i, x in enumerate(x_coords):
    for j, y in enumerate(y_coords):
        for k, z in enumerate(z_coords):
            # Position in (t, x, y, z)
            positions.append([x, y, z])
            
initial_xzs = [[1.4537, 1.4262], [0.6900, 0.3308], [1.1884, 0.3580], [0.1530, 1.9874]]
initial_xs = (torch.tensor([[xz[0], y_init, xz[1]] for xz in initial_xzs]) - 1.0) / 2.0
initial_vs = torch.tensor([[0, 8.0, 0] for xz in initial_xzs])

final_xzs = [[1.3237010240554810, -0.9951087832450867], 
             [0.9623088836669922,  -0.4938677251338959],
             [1.0516376495361328, -0.9023237228393555], 
             [-1.1842994689941406, 1.9495669603347778]]
final_xzs = torch.tensor(final_xzs)

In [6]:
BH_locs = positions
BH_masses = masses
# BH_locs = [[0.0, 0.0, 0.0]]
# BH_masses = np.array([1.]) * 2

ODEfunc = NeuralODEfunc(BH_masses_presoftplus=BH_masses, BH_positions=BH_locs, BH_masses=BH_masses, max_x=2.0)
phi, chi, alpha = ODEfunc.BrillLindquist_init()
beta = torch.zeros((ODEfunc.res, ODEfunc.res, ODEfunc.res, 3))
B = torch.zeros((ODEfunc.res, ODEfunc.res, ODEfunc.res, 3))

gamma_bar_mat_lo, _, _, _, \
Gamma, _, _, _, Gamma_bar, \
K, A_bar_mat_lo, _, _, \
_, _, _ =  ODEfunc.init_non_evolve_vars(phi)

BH_masses stats: min, max, mean: 0.0 0.20000000298023224 0.013703703880310059
Preparing Variables


## Torchdiffeq

In [7]:
from torchdiffeq import odeint_adjoint as odeint
from torchdiffeq import odeint_event

In [8]:
terminating_y = 1.0
# terminating_v = 0.5

def terminate_func(t, y_packed):
    y = y_packed[-2]
    flag = torch.ones(y.shape[0])
    for i in range(len(y)):
        if y[i][1] >= terminating_y:
            flag[i] = 0
    if torch.isnan(y_packed[-4]):
        flag[0] = 0
    if torch.isnan(y_packed[-3]):
        flag[0] = 0
    if y_packed[-4] > 5:
        flag[0] = 0
    if y_packed[-3] > 5:
        flag[0] = 0
#     for i in range(len(y[0])):
#         if torch.sum(torch.square(y[1][i][1:])) <= terminating_v:
#             flag[i] = 0
    return flag

In [21]:
hc = torch.tensor(0.0)
mc = torch.tensor(0.0)
x0 = initial_xs[0:1]
v0 = initial_vs[0:1]
y0 = (alpha, beta, chi, gamma_bar_mat_lo, Gamma_bar, A_bar_mat_lo, K, B, hc, mc, x0, v0)
t = torch.linspace(0, 0.8, 80)
# sol = odeint(ODEfunc, y0, t, rtol=1e-4, atol=1e-4)
t_final, sol = odeint_event(ODEfunc, y0, torch.tensor([0.0]), event_fn=terminate_func, 
                            reverse_time=False, odeint_interface=odeint, rtol=1e-4, atol=1e-4)

time:  0.0
Updating functions
Hamiltonian Constraint:  0.05408988893032074
Momentum Constraint:  0.0
time:  0.0007736373227089643
Updating functions
Hamiltonian Constraint:  0.05408988520503044
Momentum Constraint:  0.00043020432349294424
time:  0.007206448353827
Updating functions
Hamiltonian Constraint:  0.054089851677417755
Momentum Constraint:  0.004007362294942141
time:  0.010809672996401787
Updating functions
Hamiltonian Constraint:  0.0540897510945797
Momentum Constraint:  0.0060109407640993595
time:  0.028825793415308
Updating functions
Hamiltonian Constraint:  0.05408944562077522
Momentum Constraint:  0.01599983312189579
time:  0.03202866017818451
Updating functions
Hamiltonian Constraint:  0.05408863723278046
Momentum Constraint:  0.01775212772190571
time:  0.036032237112522125
Updating functions
Hamiltonian Constraint:  0.054088182747364044
Momentum Constraint:  0.019970476627349854
time:  0.036032237112522125
Updating functions
Hamiltonian Constraint:  0.0540892519056797
Mo

Hamiltonian Constraint:  0.053710900247097015
Momentum Constraint:  0.21498100459575653
time:  0.48833245038986206
Updating functions
Hamiltonian Constraint:  0.05370336025953293
Momentum Constraint:  0.21871423721313477
time:  0.5420424342155457
Updating functions
Hamiltonian Constraint:  0.05387181043624878
Momentum Constraint:  0.23918576538562775
time:  0.5515909194946289
Updating functions
Hamiltonian Constraint:  0.05385839566588402
Momentum Constraint:  0.24299438297748566
time:  0.5635263919830322
Updating functions
Hamiltonian Constraint:  0.05393242835998535
Momentum Constraint:  0.24835534393787384
time:  0.5635263919830322
Updating functions
Hamiltonian Constraint:  0.05402340739965439
Momentum Constraint:  0.24865829944610596
time:  0.5917581915855408
Updating functions
Hamiltonian Constraint:  0.05422527715563774
Momentum Constraint:  0.2621111571788788
time:  0.6058740615844727
Updating functions
Hamiltonian Constraint:  0.054357562214136124
Momentum Constraint:  0.27043

In [30]:
a = torch.sum(sol[-2][:, 0, [0, 1]][-1])
a.backward()

Updating functions
Hamiltonian Constraint:  0.0224942397326231
Momentum Constraint:  0.06272980570793152
Updating functions
Hamiltonian Constraint:  0.022454259917140007
Momentum Constraint:  0.06240228936076164
Updating functions
Hamiltonian Constraint:  0.022322649136185646
Momentum Constraint:  0.06056207790970802
Updating functions
Hamiltonian Constraint:  0.022367194294929504
Momentum Constraint:  0.06131574511528015
Updating functions
Hamiltonian Constraint:  0.022321823984384537
Momentum Constraint:  0.06035183370113373
Updating functions
Hamiltonian Constraint:  0.0223409291356802
Momentum Constraint:  0.05942879617214203
Updating functions
Hamiltonian Constraint:  0.022258952260017395
Momentum Constraint:  0.05685469135642052
Updating functions
Hamiltonian Constraint:  0.022322772070765495
Momentum Constraint:  0.05696970224380493
Updating functions
Hamiltonian Constraint:  0.022263692691922188
Momentum Constraint:  0.056023869663476944
Updating functions
Hamiltonian Constrain

In [13]:
ODEfunc.BH_masses_presoftplus.grad

tensor([-16.0308,  -5.2135,   7.6545,  -5.2077,   0.1766,   7.6208,   7.6599,
          7.6130,   7.6752,  -5.2099,   0.1620,   7.6160,   0.1691,  -1.7239,
          0.1825,   7.6098,   0.1762,  -5.1722,   7.6351,   7.6087,   7.6712,
          7.5985,   0.1715,  -5.1764,   7.6559,  -5.1789, -15.9804],
       dtype=torch.float64)

## Optimization

In [4]:
def loss(target, y):
    return torch.sum(torch.square(target - y))
target = torch.tensor([[0, 1, 0.]])

In [5]:
optimizer = torch.optim.Adam([ODEfunc.BH_masses_presoftplus], lr=0.01)

losses = []
lrs = []

for epoch in range(0, 600):
    optimizer.zero_grad()   
    
    phi, chi, alpha = ODEfunc.BrillLindquist_init()
    beta = torch.zeros((ODEfunc.res, ODEfunc.res, ODEfunc.res, 3))
    B = torch.zeros((ODEfunc.res, ODEfunc.res, ODEfunc.res, 3))

    gamma_bar_mat_lo, _, _, _, \
    Gamma, _, _, _, Gamma_bar, \
    K, A_bar_mat_lo, _, _, \
    _, _, _ =  ODEfunc.init_non_evolve_vars(phi)
    
    hc = torch.tensor(0.0)
    mc = torch.tensor(0.0)
    x0 = torch.tensor([[-0.47, -1, 0]])
    v0 = torch.tensor([[0, 5, 0]])
    y0 = (alpha, beta, chi, gamma_bar_mat_lo, Gamma_bar, A_bar_mat_lo, K, B, hc, mc, x0, v0)
    t = torch.linspace(0, 0.40, 20)
    y = odeint(ODEfunc, y0, t, rtol=1e-3, atol=1e-3)
    
    
    l = loss(target, y[-2][-1])
    
    
    l.backward()
    optimizer.step()
    
    losses.append(l.item())
    print(f"Epoch {epoch+1}: Loss={l:.6f}")

Preparing Variables
time:  0.0
Updating functions
Hamiltonian Constraint:  0.01783091574907303
Momentum Constraint:  0.0
time:  0.004183167591691017
Updating functions
Hamiltonian Constraint:  0.01783129759132862
Momentum Constraint:  0.0008599241264164448
time:  0.013567409478127956
Updating functions
Hamiltonian Constraint:  0.017834939062595367
Momentum Constraint:  0.002789021236822009
time:  0.020351113751530647
Updating functions
Hamiltonian Constraint:  0.017830893397331238
Momentum Constraint:  0.004184459801763296
time:  0.054269637912511826
Updating functions
Hamiltonian Constraint:  0.01782095432281494
Momentum Constraint:  0.00978583563119173
time:  0.0602995939552784
Updating functions
Hamiltonian Constraint:  0.0176173597574234
Momentum Constraint:  0.00942291785031557
time:  0.06783703714609146
Updating functions
Hamiltonian Constraint:  0.01760067045688629
Momentum Constraint:  0.010814331471920013
time:  0.06783703714609146
Updating functions
Hamiltonian Constraint:  0

Hamiltonian Constraint:  0.020477384328842163
Momentum Constraint:  0.024998238310217857
time:  0.2571505010128021
Updating functions
Hamiltonian Constraint:  0.020750708878040314
Momentum Constraint:  0.023602887988090515
time:  0.25971001386642456
Updating functions
Hamiltonian Constraint:  0.02072845585644245
Momentum Constraint:  0.023637736216187477
time:  0.2725074291229248
Updating functions
Hamiltonian Constraint:  0.020612528547644615
Momentum Constraint:  0.024120686575770378
time:  0.27478253841400146
Updating functions
Hamiltonian Constraint:  0.02055881917476654
Momentum Constraint:  0.024247964844107628
time:  0.2776263952255249
Updating functions
Hamiltonian Constraint:  0.020518533885478973
Momentum Constraint:  0.024484144523739815
time:  0.2776263952255249
Updating functions
Hamiltonian Constraint:  0.020553061738610268
Momentum Constraint:  0.02454334683716297
time:  0.2827743887901306
Updating functions
Hamiltonian Constraint:  0.020472295582294464
Momentum Constrai

Hamiltonian Constraint:  0.020528193563222885
Momentum Constraint:  0.036237042397260666
time:  0.39703822135925293
Updating functions
Hamiltonian Constraint:  0.020682882517576218
Momentum Constraint:  0.03762397915124893
time:  0.3989274501800537
Updating functions
Hamiltonian Constraint:  0.020672662183642387
Momentum Constraint:  0.03781968727707863
time:  0.4012889862060547
Updating functions
Hamiltonian Constraint:  0.020709801465272903
Momentum Constraint:  0.038170378655195236
time:  0.4012889862060547
Updating functions
Hamiltonian Constraint:  0.020762309432029724
Momentum Constraint:  0.03827263414859772
time:  0.4000000059604645
Updating functions
Hamiltonian Constraint:  0.020756138488650322
Momentum Constraint:  0.03808518871665001
time:  0.398123562335968
Updating functions
Hamiltonian Constraint:  0.02073042280972004
Momentum Constraint:  0.0378170870244503
time:  0.39465534687042236
Updating functions
Hamiltonian Constraint:  0.020687701180577278
Momentum Constraint:  

time:  0.354397177696228
Updating functions
Hamiltonian Constraint:  0.019852738827466965
Momentum Constraint:  0.03320612013339996
time:  0.36477017402648926
Updating functions
Hamiltonian Constraint:  0.020078390836715698
Momentum Constraint:  0.03398512303829193
time:  0.3638191223144531
Updating functions
Hamiltonian Constraint:  0.020055674016475677
Momentum Constraint:  0.03389612212777138
time:  0.35906389355659485
Updating functions
Hamiltonian Constraint:  0.019951604306697845
Momentum Constraint:  0.033532604575157166
time:  0.3582185208797455
Updating functions
Hamiltonian Constraint:  0.01993740350008011
Momentum Constraint:  0.03347719833254814
time:  0.35716181993484497
Updating functions
Hamiltonian Constraint:  0.01991470530629158
Momentum Constraint:  0.033405158668756485
time:  0.35716181993484497
Updating functions
Hamiltonian Constraint:  0.01991066336631775
Momentum Constraint:  0.03340156748890877
time:  0.35789474844932556
Updating functions
Hamiltonian Constrain

time:  0.3088690936565399
Updating functions
Hamiltonian Constraint:  0.01975904032588005
Momentum Constraint:  0.02875061333179474
time:  0.297335147857666
Updating functions
Hamiltonian Constraint:  0.020075708627700806
Momentum Constraint:  0.027158023789525032
time:  0.2952846586704254
Updating functions
Hamiltonian Constraint:  0.020118307322263718
Momentum Constraint:  0.026906223967671394
time:  0.29272159934043884
Updating functions
Hamiltonian Constraint:  0.020192740485072136
Momentum Constraint:  0.02651648409664631
time:  0.29272159934043884
Updating functions
Hamiltonian Constraint:  0.020222071558237076
Momentum Constraint:  0.026441117748618126
time:  0.2947368323802948
Updating functions
Hamiltonian Constraint:  0.020164625719189644
Momentum Constraint:  0.026735590770840645
time:  0.29371076822280884
Updating functions
Hamiltonian Constraint:  0.020195096731185913
Momentum Constraint:  0.02658524550497532
time:  0.2885315418243408
Updating functions
Hamiltonian Constra

...

time:  0.3638654351234436
Updating functions
Hamiltonian Constraint:  0.01877160742878914
Momentum Constraint:  0.03174151852726936
time:  0.3619801998138428
Updating functions
Hamiltonian Constraint:  0.01872960478067398
Momentum Constraint:  0.0315689742565155
time:  0.3619801998138428
Updating functions
Hamiltonian Constraint:  0.0187099426984787
Momentum Constraint:  0.031552840024232864
time:  0.3582664728164673
Updating functions
Hamiltonian Constraint:  0.018636606633663177
Momentum Constraint:  0.03128151595592499
time:  0.35640963912010193
Updating functions
Hamiltonian Constraint:  0.018594903871417046
Momentum Constraint:  0.031120428815484047
time:  0.347125381231308
Updating functions
Hamiltonian Constraint:  0.018429726362228394
Momentum Constraint:  0.030485518276691437
time:  0.34547486901283264
Updating functions
Hamiltonian Constraint:  0.01842043548822403
Momentum Constraint:  0.030393578112125397
time:  0.34341174364089966
Updating functions
Hamiltonian Constraint: 

Hamiltonian Constraint:  0.018240666016936302
Momentum Constraint:  0.028116747736930847
time:  0.3153732419013977
Updating functions
Hamiltonian Constraint:  0.0182450283318758
Momentum Constraint:  0.028075790032744408
time:  0.30983367562294006
Updating functions
Hamiltonian Constraint:  0.01831706240773201
Momentum Constraint:  0.027551718056201935
time:  0.30685579776763916
Updating functions
Hamiltonian Constraint:  0.018355047330260277
Momentum Constraint:  0.027186954393982887
time:  0.2919663190841675
Updating functions
Hamiltonian Constraint:  0.018715478479862213
Momentum Constraint:  0.0254034623503685
time:  0.28931930661201477
Updating functions
Hamiltonian Constraint:  0.01873781718313694
Momentum Constraint:  0.025149764493107796
time:  0.2860105633735657
Updating functions
Hamiltonian Constraint:  0.018818620592355728
Momentum Constraint:  0.024675438180565834
time:  0.2860105633735657
Updating functions
Hamiltonian Constraint:  0.018888387829065323
Momentum Constraint

time:  0.25397783517837524
Updating functions
Hamiltonian Constraint:  0.01928148791193962
Momentum Constraint:  0.021739214658737183
time:  0.25290486216545105
Updating functions
Hamiltonian Constraint:  0.019286448135972023
Momentum Constraint:  0.021720759570598602
time:  0.25156369805336
Updating functions
Hamiltonian Constraint:  0.019292008131742477
Momentum Constraint:  0.02169596217572689
time:  0.25156369805336
Updating functions
Hamiltonian Constraint:  0.019291575998067856
Momentum Constraint:  0.021692609414458275
time:  0.2526315748691559
Updating functions
Hamiltonian Constraint:  0.019286978989839554
Momentum Constraint:  0.021712055429816246
time:  0.2524966895580292
Updating functions
Hamiltonian Constraint:  0.0192875973880291
Momentum Constraint:  0.021709561347961426
time:  0.2499338686466217
Updating functions
Hamiltonian Constraint:  0.019302133470773697
Momentum Constraint:  0.02166876383125782
time:  0.24858501553535461
Updating functions
Hamiltonian Constraint:

Hamiltonian Constraint:  0.01888125017285347
Momentum Constraint:  0.02051410637795925
time:  0.17966212332248688
Updating functions
Hamiltonian Constraint:  0.018812986090779305
Momentum Constraint:  0.02038608305156231
time:  0.17966212332248688
Updating functions
Hamiltonian Constraint:  0.01880577951669693
Momentum Constraint:  0.020373569801449776
time:  0.20715071260929108
Updating functions
Hamiltonian Constraint:  0.019175738096237183
Momentum Constraint:  0.021219616755843163
time:  0.20546291768550873
Updating functions
Hamiltonian Constraint:  0.019157549366354942
Momentum Constraint:  0.02117662876844406
time:  0.1970239281654358
Updating functions
Hamiltonian Constraint:  0.01907060109078884
Momentum Constraint:  0.020962804555892944
time:  0.19552364945411682
Updating functions
Hamiltonian Constraint:  0.019054623320698738
Momentum Constraint:  0.020921045914292336
time:  0.1936483383178711
Updating functions
Hamiltonian Constraint:  0.019029200077056885
Momentum Constrai

KeyboardInterrupt: 

In [30]:
mass_try2 = []

In [35]:
mass_try2.append(np.load("run2/epoch4.npy"))

In [100]:
masses_try = np.load("RK4/run1/epoch4.npy")
masses_try2 = np.load("RK4/run2/epoch4.npy")


In [101]:
delta = masses_try[0:100] - masses_try2[0:100]
delta_norm = np.square(delta)
delta_norm = np.sqrt(np.sum(delta_norm, axis=-1))
# delta_norm = np.square(np.random.rand(27) * 1e-5)
# delta_norm = np.sqrt(np.sum(delta_norm, axis=-1))
delta_norm_0 = np.square(np.random.rand(27) * 1e-5)
delta_norm_0 = np.sqrt(np.sum(delta_norm_0, axis=-1))

In [102]:
growth = delta_norm / delta_norm[0]